# Web Browsing

###  **Full Browsing**

In [1]:
from warnings import warn

from aiohttp import ClientSession
from lxml import html
from fastapi.concurrency import run_in_threadpool

from app.shared import Shared
from app.utils.chat.chains import Chains
from app.common.constants import JsonTemplates, QueryTemplates, JsonTemplates
from app.utils.logger import CustomLogger, LoggingConfig

logger = CustomLogger(
    "Browsing", logging_config=LoggingConfig(file_log_name="./logs/notebook.log")
)


ddg = Shared().duckduckgo
query = "Explain to me what the vision pro is that Apple released."

In [2]:
query_to_search_json = await Chains.aget_json(
    query_template=JsonTemplates.QUERY__GET_QUERY_FOR_WEB_BROWSING,
    query=query,
)
if (
    not isinstance(query_to_search_json, dict)
    or "query" not in query_to_search_json
):
    warn(f"No query template found for query: {query}")
    query_to_search = query
else:
    query_to_search = query_to_search_json["query"]
print(query_to_search)

Apple vision pro explanation


In [3]:
async def scrolling(
    link: str,
    tokens_per_chunk: int,
    chunk_overlap: int,
) -> str | None:
    async with ClientSession() as session:
        res = await session.get(link)
        paragraphs = html.fromstring(await res.text()).xpath("//p")
        scrollable_contents: list[str] = Shared().token_text_splitter.split_text(
            "\n".join([p.text_content().strip() for p in paragraphs]),
            tokens_per_chunk=tokens_per_chunk,
            chunk_overlap=chunk_overlap,
        )
        for scrollable_content in scrollable_contents:
            scrollable_content = scrollable_content.strip()
            logger.info(f"Reading content: {scrollable_content}")
            answerable_or_not_json = await Chains.aget_json(
                query_template=JsonTemplates.CONTEXT_QUERY__ANSWERABLE_OR_NOT,
                context=scrollable_content,
                question=query,
            )
            logger.info(f"answerable_or_not_json: {answerable_or_not_json}")
            if not isinstance(
                answerable_or_not_json, dict
            ) or answerable_or_not_json.get("answerable") not in (True, False):
                logger.info("Reading content failed.")
                continue
            if answerable_or_not_json["answerable"]:
                logger.info("Feedback bot decided to finish browsing.")
                return scrollable_content
            else:
                continue
        logger.info("This link is not sufficient to answer the user's question.")
        return None

In [4]:
async def full_browsing(
    query: str,
    tokens_per_chunk: int,
    chunk_overlap: int,
) -> str | None:
    query_to_search_json = await Chains.aget_json(
        query_template=JsonTemplates.QUERY__GET_QUERY_FOR_WEB_BROWSING,
        query=query,
    )
    if (
        not isinstance(query_to_search_json, dict)
        or "query" not in query_to_search_json
    ):
        query_to_search = query
    else:
        query_to_search = query_to_search_json["query"]

    snippets_with_link: dict[str, str] = await run_in_threadpool(
        ddg.formatted_results_with_link, query=query_to_search
    )

    while snippets_with_link:
        action_and_link_json = await Chains.aget_json(
            query_template=JsonTemplates.CONTEXT_QUERY__CLICK_LINK_OR_FINISH,
            query=query,
            context="\n\n".join(snippets_with_link.values()),
        )
        logger.info(f"action_and_link_json: {action_and_link_json}")
        if (
            not isinstance(action_and_link_json, dict)
            or (action_and_link_json.get("action") not in ("click", "finish"))
            or (
                action_and_link_json.get("action") == "link"
                and action_and_link_json.get("link") not in snippets_with_link
            )
        ):
            logger.info("Failed browsing.")
            return None
        if action_and_link_json.get("action") == "finish":
            logger.info("We've got the answer!")
            return "\n\n".join(snippets_with_link.values())
        snippets_with_link.pop(action_and_link_json["link"])
        scroll_result: str | None = await scrolling(
            link=action_and_link_json["link"],
            tokens_per_chunk=tokens_per_chunk,
            chunk_overlap=chunk_overlap,
        )
        if scroll_result is not None:
            logger.info("We've got the answer!")
            return scroll_result
        else:
            logger.info("We still don't have the answer.")
            continue

In [7]:
browsing_result = (
    await full_browsing(
        query=query,
        tokens_per_chunk=1024,
        chunk_overlap=256,
    ),
)

[2023-06-13 22:25:19,242] Browsing:INFO - action_and_link_json: {'action': 'click', 'link': 'https://www.techradar.com/news/apple-vision-pro-everything-we-know'}
[2023-06-13 22:25:19,475] Browsing:INFO - Reading content: When you purchase through links on our site, we may earn an affiliate commission. Here’s how it works.
The Apple Vision Pro AR and VR headset is a reality
After years of rumors, leaks, and speculation, Apple has finally unveiled the Vision Pro, its first AR and VR headset.
-Mixed reality headset
-Dual M2 and R1 chip setup
-4K resolution per eye
-No controllers, uses hand tracking and voice inputs
-External battery pack
-Two-hour battery life
-Starts at $3,499 (around £2,800 / AU$5,300)
-Runs on visionOS
The announcement came as the 'One more thing' of Apple's WWDC 2023 event, at which it also unveiled a new 15-inch MacBook Air, new features coming with iOS 17, and a new M2 Ultra processor, among other reveals.
Many of the leaks we'd heard proved to be correct, although

In [6]:
llm = Shared().openai_llm
llm_output = await llm.apredict(  # type: ignore
    QueryTemplates.CONTEXT_QUESTION__DEFAULT.format(
        context=browsing_result, question=query
    )
)
print(llm_output)

Apple released the Vision Pro, which is a revolutionary spatial computer that seamlessly blends digital content with the physical world. It features an ultra-high-resolution display system that packs 23 million pixels across two displays, and custom Apple silicon in a unique dual-chip design to ensure every experience feels like it’s taking place in front of the user’s eyes in real-time. The Vision Pro introduces a fully three-dimensional user interface controlled by the most natural and intuitive inputs possible — a user’s eyes, hands, and voice. It runs on visionOS, the world’s first spatial operating system. It unlocks incredible experiences for users and exciting new opportunities for developers. The device brings a new dimension to powerful, personal computing by changing the way users interact with their favorite apps, capture and relive memories, enjoy stunning TV shows and movies, and connect with others in FaceTime. It offers an infinite canvas for apps at work and at home, en